# Jupyter client 

`jupyter_client` is a package that implements a system for managing jupyter kernels. See the [official documentation](https://jupyter-client.readthedocs.io/en/latest/index.html). Actually, it provides access to the programming management of jupyter.

## CLI

`jupyter-kernel` tool comes with `jupyter_client` package.

---

It confuses me that `jupyter_client` provides the `jupyter-kernel` CLI tool. If you have the same doubts, the following cells will prove it.

The code in the following cell shows the file to which `jupyter-kernel` refers in the system.

In [1]:
!which jupyter-kernel

/home/fedor/Documents/knowledge/venv/bin/jupyter-kernel


And behind it is Python code that refers to `jupyter_client.kernelapp.main`.

In [2]:
!cat $(which jupyter-kernel)

#!/home/fedor/Documents/knowledge/venv/bin/python
# -*- coding: utf-8 -*-
import re
import sys
from jupyter_client.kernelapp import main
if __name__ == '__main__':
    sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])
    sys.exit(main())


## Running kernel

## Creating kernel